In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from ipywidgets.widgets import interact

In [2]:
class Ut8channels:
    '''
    Class that read unstructured bytes sequence and outputs them in a structured way.
    ''' 
    SENSORS_NUMBER = 8  
    HEADER_BYTES = 4
    SENSOR_BYTES = 1
    CTP_BYTES = 3
    ENCODER_BYTES = 3
    DATA_BYTES = 1017 # for 1 kB A-scan length
    # DATA_BYTES = 2041 # for 2 kB A-scan length
    MAX_BYTE_VALUE = 0xFF
    MIN_BYTE_VALUE = 0x00

    def __init__(self, filename):
            self.filename = filename
            self._data = self.read_utd()

    def read_utd(self):
        '''
        This function reads the ut raw file and returns a list of lists with a-scans.

        '''
        data = [[] for _ in range(Ut8channels.SENSORS_NUMBER)]
        raw_data = open(self.filename, 'rb')
        while True:  
            header_bytes = raw_data.read(Ut8channels.HEADER_BYTES)
            sensor_bytes = raw_data.read(Ut8channels.SENSOR_BYTES)
            ctp_bytes = raw_data.read(Ut8channels.CTP_BYTES) 
            encoder_bytes = raw_data.read(Ut8channels.ENCODER_BYTES)
            data_bytes = raw_data.read(Ut8channels.DATA_BYTES)
            data_array = np.frombuffer(data_bytes, dtype = np.uint8) # Interpret a buffer (bytes) as a 1-dimensional array
            output_array = (2*((data_array - Ut8channels.MIN_BYTE_VALUE)/
                (Ut8channels.MAX_BYTE_VALUE - Ut8channels.MIN_BYTE_VALUE))-1).round(2) # Data normalization [-1, 1]
            try:
                data[ord(sensor_bytes)].append(output_array) # Get the number that represents the sensor
            except:
                break

        # fill missing values with array of zeros so all sensors have the same number of ascans
        max_length = max([len(sensor) for sensor in data]) 
        fill = [np.zeros(Ut8channels.DATA_BYTES)] * max_length
        data = [sensor[:max_length] + fill[len(sensor):] for sensor in data] 

        df = pd.DataFrame(data).transpose()
        df.columns = list(range(1, Ut8channels.SENSORS_NUMBER+1)) # 1 to 8
        # df = df.fillna(value=0) doesn't fill with np array --'
        return df

    @property
    def data(self):
        return self._data

    def plot_ascan(self, sensor, index, show_peaks=False):
        plt.figure(figsize=(6,4), dpi=150)
        ascan = self.data[sensor][index]
        plt.ylim(-1.2, 1.2)
        color = None if any(ascan) else 'red' # if ascan was filled with zeros to equalize numbers of ascans, than it is red
        plt.plot(ascan, color=color)
        if show_peaks:
            peaks, _ = scipy.signal.find_peaks(ascan, distance=50, prominence=0.2)
            plt.scatter(peaks, ascan[peaks], marker="x", color='orange')    

    def plot_interactive_ascan(self):
        @interact(sensor=(1, Ut8channels.SENSORS_NUMBER), index=(0, len(self.data)-1), show_peaks=(0,1))
        def _plot_interactive_ascan(sensor, index, show_peaks):
            self.plot_ascan(sensor=sensor, index=index, show_peaks=show_peaks)

In [3]:
ut = Ut8channels('dataset/RET-001/Face1/RET-001-Face1-001.utd')

In [4]:
# ut.data

In [5]:
# ut.plot_ascan(sensor=6, index=62, show_peaks=True)

In [6]:
ut.plot_interactive_ascan()

interactive(children=(IntSlider(value=4, description='sensor', max=8, min=1), IntSlider(value=62, description=…